Defect Detection using Real Time Detection Transformer

In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="UjVLCDuAMxom6CBhWdDX")
project = rf.workspace("defect-detection-guglj").project("defect_detection_new-wkheb")
version = project.version(3)
dataset = version.download("coco")

KeyboardInterrupt: 

In [ ]:
from ultralytics import RTDETR
import torch

model = RTDETR("test.yaml")
dummy = torch.rand(1, 3, 640, 640)
with torch.no_grad():
    out = model(dummy)
    print(out) 

In [ ]:
from ultralytics import RTDETR
import torch

# Load your RT-DETR DenseNet model
model = RTDETR("test.yaml")

# The first module is the backbone
backbone = model.model.model[0]

# Dummy input
x = torch.randn(1, 3, 640, 640)

# Run through the backbone to see feature maps
with torch.no_grad():
    features = backbone(x)
    print("\n--- Backbone outputs ---")
    for i, f in enumerate(features):
        print(f"Index {i}: {tuple(f.shape)}")


In [3]:
import torch
from rtdetr_custom import RTDETRCustom

# --- Instantiate and test ---
device = "cuda" if torch.cuda.is_available() else "cpu"
model = RTDETRCustom().to(device)
model.eval()

dummy = torch.randn(1, 3, 640, 640).to(device)
with torch.no_grad():
    out = model(dummy)

print("\n✅ Forward succeeded.")
if isinstance(out, tuple):
    print("Output is tuple of length:", len(out))
    for i, o in enumerate(out):
        if torch.is_tensor(o):
            print(f"  [{i}] Tensor shape:", tuple(o.shape))
        elif isinstance(o, (list, tuple)):
            print(f"  [{i}] nested len:", len(o))
else:
    print("Output type:", type(out))


Loading base RT-DETR model...
WARNING no model scale passed. Assuming scale='l'.
Replacing backbone with SqueezeNet...
Loaded pretrained weights for squeezenet1_1
Adding AIFI neck...
Configuring RTDETRDecoder head...
✅ Custom RTDETR model initialized successfully.
Model parameters: 10302534

✅ Forward succeeded.
Output is tuple of length: 2
  [0] Tensor shape: (1, 300, 10)
  [1] nested len: 5


In [ ]:
import torch
from torchvision.models import densenet121

m = densenet121(weights="DEFAULT").features.eval()
x = torch.randn(1, 3, 640, 640)
out = []
for i, (name, layer) in enumerate(m._modules.items()):
    x = layer(x)
    if name in ["denseblock1", "denseblock2", "denseblock3", "denseblock4"]:
        out.append((name, x.shape))
print(out)


In [ ]:
import torch
import torchvision.models as models
from torch import nn

# Load pretrained EfficientNet-V2-M backbone
m = models.efficientnet_v2_m(weights=models.EfficientNet_V2_M_Weights.DEFAULT)
m.eval()

# Create a feature extractor that outputs intermediate feature maps
# This is available in torchvision >= 0.13
from torchvision.models.feature_extraction import create_feature_extractor

# Specify which layers you want to extract
return_nodes = {
    'features.3': 'stage1',   # stride 8   -> (80, 80)
    'features.6': 'stage2',   # stride 16  -> (160, 40)
    'features.8': 'stage3',   # stride 32  -> (1280, 20)
}

feature_extractor = create_feature_extractor(m, return_nodes=return_nodes)

# Test with dummy input
x = torch.rand(1, 3, 640, 640)
with torch.no_grad():
    out = feature_extractor(x)

# Print each output shape
for k, v in out.items():
    print(k, v.shape)


In [ ]:
import torch
import torchvision.models as models
from torch import nn

# Load pretrained EfficientNetV2-M backbone
model = models.efficientnet_b0(weights="DEFAULT")

# Set model to eval mode
model.eval()

# Dummy input tensor
x = torch.randn(1, 3, 640, 640)

# Dictionary to store shapes
shapes = {}

# Hook function to record tensor shapes
def hook_fn(name):
    def hook(module, input, output):
        if isinstance(output, torch.Tensor):
            shapes[name] = tuple(output.shape)
        elif isinstance(output, (list, tuple)):
            shapes[name] = [tuple(o.shape) for o in output if isinstance(o, torch.Tensor)]
    return hook

# Register forward hooks for all layers
hooks = []
for name, module in model.named_modules():
    if isinstance(module, (nn.Conv2d, nn.BatchNorm2d, nn.ReLU, nn.SiLU, nn.Sequential, nn.AdaptiveAvgPool2d)):
        hooks.append(module.register_forward_hook(hook_fn(name)))

# Run forward pass
with torch.no_grad():
    _ = model(x)

# Print layer names and output shapes
print("\n📊 EfficientNetV2-M Layer Output Shapes:\n")
for name, shape in shapes.items():
    print(f"{name:<40} -> {shape}")

# Remove hooks
for h in hooks:
    h.remove()


In [1]:
import torch
from ultralytics import RTDETR

model = RTDETR("rtdetr_densenet121.yaml")

dummy = torch.rand(1, 3, 640, 640)
with torch.no_grad():
    out = model(dummy)
    print(out)


0: 640x640 300 2s, 58.8ms
Speed: 0.7ms preprocess, 58.8ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5'}
obb: None
orig_img: array([[[184, 123,  30],
        [ 55,   6, 174],
        [ 96, 133, 192],
        ...,
        [ 57,  63,  94],
        [238,  72, 114],
        [145, 145, 178]],

       [[128, 115,  39],
        [168, 171, 241],
        [125,  92, 254],
        ...,
        [227, 191, 123],
        [205, 109,  68],
        [226, 225,  14]],

       [[ 80, 138, 126],
        [112,  60,  36],
        [245, 184, 225],
        ...,
        [186, 186,  16],
        [ 59,  52,  85],
        [ 81, 173, 188]],

       ...,

       [[180,  17,  43],
        [ 46, 130, 246],
        [176, 147,  78],
        ...,
        [  9,  55, 175],
        [115, 169, 204],
        [ 73,  78,

In [2]:
from cspdarknet import CspDarknet53
import ultralytics.nn.tasks as tasks
from ultralytics import RTDETR
from ultralytics.nn.tasks import parse_model
import ultralytics.nn.tasks as tasks
import torch
import torch.nn as nn
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
tasks.__dict__["CspDarknet53"] = CspDarknet53

class FeatureSelector(nn.Module):
    def __init__(self, index, in_channels_list):
        super().__init__()
        self.index = index
        # ✅ Tell the parser what the output channels of *this* module will be
        self.out_channels = in_channels_list[self.index]
    
    def forward(self, x):
        # x will be the list [P3, P4, P5]
        return x[self.index]
tasks.__dict__["FeatureSelector"] = FeatureSelector


model_csp = RTDETR("rtdetr_cspdarknet53.yaml").to(device)


dummy = torch.cuda.FloatTensor(1, 3, 640, 640).to(device)
with torch.no_grad():
    out = model_csp(dummy)
    print(out)


Using device: cuda
WARNING no model scale passed. Assuming scale='l'.
CSPDarknet53 feature channels: [128, 256, 512]



C:\Users\USER X\AppData\Local\Temp\ipykernel_43556\4240581470.py:28: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\tensor\python_tensor.cpp:80.)
  dummy = torch.cuda.FloatTensor(1, 3, 640, 640).to(device)


RuntimeError: Given groups=1, weight of size [256, 3, 1, 1], expected input[1, 128, 160, 160] to have 3 channels, but got 128 channels instead